In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

from datetime import datetime, date, time

In [2]:
from google.cloud import bigquery
from google.oauth2 import service_account
path_to_private_key = './ieso-dashboard-c639f1a39298.json'
credentials = service_account.Credentials.from_service_account_file(
    path_to_private_key,
    scopes=['https://www.googleapis.com/auth/cloud-platform'],
)

In [3]:
old_ade=pd.read_csv('adequacy_202209052348.csv')

In [4]:
query_string="SELECT * FROM `ieso-dashboard.Adequacy2.Adequacy2_table` ORDER BY mkt_date, mkt_he;"
gbq_df = pd.read_gbq(query_string, credentials=credentials)

In [12]:
pd.set_option('display.max_columns', None)
gbq_df.tail()

,mkt_datetime,mkt_date,mkt_he,dacp_flag,created_at,ont_cap,ont_ene,int_nuc_cap,int_nuc_out,int_nuc_off,int_nuc_sch,int_gas_cap,int_gas_out,int_gas_off,int_gas_sch,int_hyd_cap,int_hyd_out,int_hyd_for,int_hyd_off,int_hyd_sch,int_win_cap,int_win_out,int_win_for,int_win_sch,int_sol_cap,int_sol_out,int_sol_for,int_sol_sch,int_bio_cap,int_bio_out,int_bio_off,int_bio_sch,int_oth_cap,int_oth_out,int_oth_off,int_oth_sch,int_tot_out,int_tot_off,int_tot_sch,imp_man_off,imp_man_sch,imp_min_off,imp_min_sch,imp_mic_off,imp_mic_sch,imp_new_off,imp_new_sch,imp_que_off,imp_que_sch,imp_tot_off,imp_tot_sch,imp_tot_est,imp_tot_cap,ont_bot_cap,ont_reg,ont_tot_sup,ont_for_dem,ont_peak_dem,ont_avg_dem,ont_emb_wind,ont_emb_sol,ont_disp_cap,ont_disp_bid,ont_disp_on,ont_disp_off,ont_hdr_bid,ont_hdr_sch,ont_hdr_cur,exp_man_bid,exp_man_sch,exp_min_bid,exp_min_sch,exp_mic_bid,exp_mic_sch,exp_new_bid,exp_new_sch,exp_que_bid,exp_que_sch,exp_tot_bid,exp_tot_sch,exp_tot_cap,grh_tot,grh_min_10n,grh_min_10s,grh_lfu,grh_add,total_req,cap_excess,energy_excess,offered_cap_excess,res_not_sch,imp_not_sch
3331,2022-10-11 19:00:00,2022-10-11,20,None,2022-09-07 09:06:08+00:00,37630,21729,13081,6601,None,None,10679,2034,None,None,8118,1835,4459,None,None,4943,3732,1211,None,478,478,0,None,315,283,None,None,17,0,None,None,15389,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2000,250,1264,100,23126,16753,None,16753,None,None,513,125,None,None,500,None,None,None,None,None,None,None,None,None,None,None,None,None,None,42,1418,945,236,0,907,18495,4631,5601,None,None,None
3332,2022-10-11 20:00:00,2022-10-11,21,None,2022-09-07 09:06:08+00:00,37630,21610,13081,6601,None,None,10679,2034,None,None,8118,1833,4340,None,None,4943,3732,1211,None,478,478,0,None,315,283,None,None,17,0,None,None,15387,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2000,250,1264,100,23128,16191,None,16191,None,None,513,125,None,None,500,None,None,None,None,None,None,None,None,None,None,None,None,None,None,42,1418,945,236,0,907,17934,5194,6043,None,None,None
3333,2022-10-11 21:00:00,2022-10-11,22,None,2022-09-07 09:06:08+00:00,37630,21700,13081,6601,None,None,10679,2034,None,None,8118,1833,4580,None,None,4943,3732,1211,None,478,478,0,None,315,283,None,None,17,0,None,None,15387,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2000,100,1264,100,22978,15445,None,15445,None,None,513,125,None,None,0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,42,1418,945,236,0,907,17687,5291,6380,None,None,None
3334,2022-10-11 22:00:00,2022-10-11,23,None,2022-09-07 09:06:08+00:00,37630,21441,13081,6601,None,None,10679,1714,None,None,8118,1840,4001,None,None,4943,3732,1211,None,478,478,0,None,315,283,None,None,17,0,None,None,15080,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2000,100,1264,100,23285,14057,None,14057,None,None,513,125,None,None,0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,42,1418,945,236,0,907,16299,6986,7509,None,None,None
3335,2022-10-11 23:00:00,2022-10-11,24,None,2022-09-07 09:06:08+00:00,37630,21216,13081,6601,None,None,10679,1714,None,None,8118,1799,3782,None,None,4943,3732,1211,None,478,478,0,None,315,289,None,None,17,0,None,None,15046,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2000,100,1264,100,23319,12992,None,12992,None,None,513,125,None,None,0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,42,1418,945,236,0,907,15234,8085,8349,None,None,None


In [14]:
hour_col = gbq_df['mkt_he'].apply(lambda x: str(datetime.strptime(str(x-1),'%H').time()))
# hour_col = gbq_df['mkt_he'].apply(lambda x: str(x-1))
date_col = gbq_df['mkt_date'].apply(lambda x: str(x))
date_hour=pd.to_datetime(date_col + hour_col, format='%m/%d/%Y %H:%M')
# pd.to_datetime(gbq_df['mkt_date'] +' '+ hour_col)

ValueError: time data '2022-05-2600:00:00' does not match format '%m/%d/%Y %H:%M' (match)

In [7]:
gbq_df.insert(loc = 0,
          column = 'mkt_datetime',
          value = date_hour)

In [8]:
gbq_df.tail()

,mkt_datetime,mkt_date,mkt_he,dacp_flag,created_at,ont_cap,ont_ene,int_nuc_cap,int_nuc_out,int_nuc_off,int_nuc_sch,int_gas_cap,int_gas_out,int_gas_off,int_gas_sch,int_hyd_cap,int_hyd_out,int_hyd_for,int_hyd_off,int_hyd_sch,int_win_cap,int_win_out,int_win_for,int_win_sch,int_sol_cap,int_sol_out,int_sol_for,int_sol_sch,int_bio_cap,int_bio_out,int_bio_off,int_bio_sch,int_oth_cap,int_oth_out,int_oth_off,int_oth_sch,int_tot_out,int_tot_off,int_tot_sch,imp_man_off,imp_man_sch,imp_min_off,imp_min_sch,imp_mic_off,imp_mic_sch,imp_new_off,imp_new_sch,imp_que_off,imp_que_sch,imp_tot_off,imp_tot_sch,imp_tot_est,imp_tot_cap,ont_bot_cap,ont_reg,ont_tot_sup,ont_for_dem,ont_peak_dem,ont_avg_dem,ont_emb_wind,ont_emb_sol,ont_disp_cap,ont_disp_bid,ont_disp_on,ont_disp_off,ont_hdr_bid,ont_hdr_sch,ont_hdr_cur,exp_man_bid,exp_man_sch,exp_min_bid,exp_min_sch,exp_mic_bid,exp_mic_sch,exp_new_bid,exp_new_sch,exp_que_bid,exp_que_sch,exp_tot_bid,exp_tot_sch,exp_tot_cap,grh_tot,grh_min_10n,grh_min_10s,grh_lfu,grh_add,total_req,cap_excess,energy_excess,offered_cap_excess,res_not_sch,imp_not_sch
3331,2022-10-11 19:00:00,2022-10-11,20,None,2022-09-07 09:06:08+00:00,37630,21729,13081,6601,None,None,10679,2034,None,None,8118,1835,4459,None,None,4943,3732,1211,None,478,478,0,None,315,283,None,None,17,0,None,None,15389,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2000,250,1264,100,23126,16753,None,16753,None,None,513,125,None,None,500,None,None,None,None,None,None,None,None,None,None,None,None,None,None,42,1418,945,236,0,907,18495,4631,5601,None,None,None
3332,2022-10-11 20:00:00,2022-10-11,21,None,2022-09-07 09:06:08+00:00,37630,21610,13081,6601,None,None,10679,2034,None,None,8118,1833,4340,None,None,4943,3732,1211,None,478,478,0,None,315,283,None,None,17,0,None,None,15387,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2000,250,1264,100,23128,16191,None,16191,None,None,513,125,None,None,500,None,None,None,None,None,None,None,None,None,None,None,None,None,None,42,1418,945,236,0,907,17934,5194,6043,None,None,None
3333,2022-10-11 21:00:00,2022-10-11,22,None,2022-09-07 09:06:08+00:00,37630,21700,13081,6601,None,None,10679,2034,None,None,8118,1833,4580,None,None,4943,3732,1211,None,478,478,0,None,315,283,None,None,17,0,None,None,15387,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2000,100,1264,100,22978,15445,None,15445,None,None,513,125,None,None,0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,42,1418,945,236,0,907,17687,5291,6380,None,None,None
3334,2022-10-11 22:00:00,2022-10-11,23,None,2022-09-07 09:06:08+00:00,37630,21441,13081,6601,None,None,10679,1714,None,None,8118,1840,4001,None,None,4943,3732,1211,None,478,478,0,None,315,283,None,None,17,0,None,None,15080,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2000,100,1264,100,23285,14057,None,14057,None,None,513,125,None,None,0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,42,1418,945,236,0,907,16299,6986,7509,None,None,None
3335,2022-10-11 23:00:00,2022-10-11,24,None,2022-09-07 09:06:08+00:00,37630,21216,13081,6601,None,None,10679,1714,None,None,8118,1799,3782,None,None,4943,3732,1211,None,478,478,0,None,315,289,None,None,17,0,None,None,15046,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2000,100,1264,100,23319,12992,None,12992,None,None,513,125,None,None,0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,42,1418,945,236,0,907,15234,8085,8349,None,None,None


In [9]:
ind_var = ['mkt_datetime','int_nuc_sch', 'int_gas_sch',  'int_hyd_sch', 'int_win_sch', 'imp_mic_sch', 'imp_new_sch', 'imp_que_sch', 'ont_avg_dem', 'exp_mic_sch', 'exp_new_sch', 'exp_que_sch']
X = gbq_df[ind_var]

In [10]:
X.tail()

,mkt_datetime,int_nuc_sch,int_gas_sch,int_hyd_sch,int_win_sch,imp_mic_sch,imp_new_sch,imp_que_sch,ont_avg_dem,exp_mic_sch,exp_new_sch,exp_que_sch
3331,2022-10-11 19:00:00,None,None,None,None,None,None,None,16753,None,None,None
3332,2022-10-11 20:00:00,None,None,None,None,None,None,None,16191,None,None,None
3333,2022-10-11 21:00:00,None,None,None,None,None,None,None,15445,None,None,None
3334,2022-10-11 22:00:00,None,None,None,None,None,None,None,14057,None,None,None
3335,2022-10-11 23:00:00,None,None,None,None,None,None,None,12992,None,None,None


In [13]:
old_ade.head()

,mkt_date,mkt_he,mkt_datetime,utc_datetime,dacp_flag,file_ver,updated_at,ont_cap,ont_ene,int_nuc_cap,int_nuc_out,int_nuc_off,int_nuc_sch,int_gas_cap,int_gas_out,int_gas_off,int_gas_sch,int_hyd_cap,int_hyd_out,int_hyd_for,int_hyd_off,int_hyd_sch,int_win_cap,int_win_out,int_win_for,int_win_sch,int_sol_cap,int_sol_out,int_sol_for,int_sol_sch,int_bio_cap,int_bio_out,int_bio_off,int_bio_sch,int_oth_cap,int_oth_out,int_oth_off,int_oth_sch,int_tot_out,int_tot_off,int_tot_sch,imp_man_off,imp_man_sch,imp_min_off,imp_min_sch,imp_mic_off,imp_mic_sch,imp_new_off,imp_new_sch,imp_que_off,imp_que_sch,imp_tot_off,imp_tot_sch,imp_tot_est,imp_tot_cap,ont_bot_cap,ont_reg,ont_tot_sup,ont_for_dem,ont_peak_dem,ont_avg_dem,ont_emb_wind,ont_emb_sol,ont_disp_cap,ont_disp_bid,ont_disp_on,ont_disp_off,ont_hdr_bid,ont_hdr_sch,ont_hdr_cur,exp_man_bid,exp_man_sch,exp_min_bid,exp_min_sch,exp_mic_bid,exp_mic_sch,exp_new_bid,exp_new_sch,exp_que_bid,exp_que_sch,exp_tot_bid,exp_tot_sch,exp_tot_cap,grh_tot,grh_min_10n,grh_min_10s,grh_lfu,grh_add,total_req,cap_excess,energy_excess,offered_cap_excess,res_not_sch,imp_not_sch
0,1/1/2020,1,1/1/2020 0:00,1/1/2020 5:00,PD,127,1/1/2020 23:49,36306,27771,13032,1484,12220,11554,9829,547,3306,216,8058,1042,4235,6098,2857,4483,1105,3378,2113,424,424,0,0,480,251,14,10,6,0,0,0,5464,25016,16749,355,64,160,0,409,0,124,0,1421,9,2469,73,0,0,800,100,29941,13305,13414,13305,209,0,483,121,-118,3,7.0,-7.0,0.0,633,-202,226,0,1374,-800,1926,-1285,2073,-1273,6232,-3560,629,1418,945,236,0,0,15349.0,14592.0,14466,12780.0,8391.0,2396
1,1/1/2020,2,1/1/2020 1:00,1/1/2020 6:00,PD,127,1/1/2020 23:49,36306,27468,13032,1484,12220,11554,9829,547,3306,216,8058,1042,3946,6103,2834,4483,1118,3365,1595,424,424,0,0,480,251,14,10,6,0,0,0,5477,25008,16209,355,64,160,0,409,0,124,0,1421,9,2469,73,0,0,800,100,29928,12870,13088,12870,214,0,483,121,-118,3,NaN,NaN,NaN,507,-201,245,0,1363,-800,1926,-1285,2092,-1273,6133,-3559,629,1418,945,236,0,0,NaN,NaN,14598,NaN,NaN,2396
2,1/1/2020,3,1/1/2020 2:00,1/1/2020 7:00,PD,127,1/1/2020 23:49,36306,27115,13032,1484,12220,11554,9829,547,3306,216,8058,1042,3906,6108,2834,4483,1432,3051,1506,424,424,0,0,480,251,14,10,6,0,0,0,5784,24700,16120,355,111,160,0,409,0,124,0,1421,9,2469,120,0,0,800,100,29621,12512,12691,12512,195,0,483,121,-118,3,NaN,NaN,NaN,475,-245,245,0,1423,-800,1832,-1390,2119,-1273,6094,-3708,629,1418,945,236,0,0,NaN,NaN,14603,NaN,NaN,2349
3,1/1/2020,4,1/1/2020 3:00,1/1/2020 8:00,PD,127,1/1/2020 23:49,36306,27083,13032,1484,12220,11554,9829,547,3306,216,8058,1042,3906,6108,2835,4483,1464,3019,1592,424,424,0,0,480,251,14,13,6,0,0,0,5816,24667,16210,355,125,160,0,409,0,124,0,1421,9,2469,134,0,0,800,100,29590,12370,12441,12370,186,0,483,161,-161,0,0.0,0.0,0.0,475,-255,245,0,1413,-800,1793,-1443,2119,-1273,6045,-3771,629,1418,945,236,0,0,14417.0,15173.0,14713,13437.0,8618.0,2335
4,1/1/2020,5,1/1/2020 4:00,1/1/2020 9:00,PD,127,1/1/2020 23:49,36306,27014,13032,1484,12220,11554,9829,547,3306,216,8058,1042,3928,6111,2911,4483,1554,2929,1479,424,424,0,0,480,251,14,13,6,0,0,0,5904,24580,16173,355,45,160,0,409,0,124,0,1421,9,2469,54,0,0,800,100,29501,12469,12527,12469,186,0,483,161,-161,0,0.0,0.0,0.0,377,-174,245,-1,1423,-800,1867,-1440,2119,-1273,6031,-3688,629,1418,945,236,0,0,14516.0,14985.0,14545,13264.0,8567.0,2415
